### Merging data **by Larissa**
Here we will merge the following cleaned datasets:
- prescription from Florida, Kenntucky, North Carolina, 
- death data from all states
- census data from all states

---

Import first dataset containing prescription data

In [28]:
import pandas as pd
df_prescriptions = pd.read_csv('https://raw.githubusercontent.com/MIDS-at-Duke/pds2021-opioids-team-three/main/20_intermediate/prescriptions?token=AI3GZDZGWG25Z2T5V2RXQGDBRO3WS', index_col=0)
df_prescriptions.head()

,BUYER_COUNTY,T_DATE,MME,State
0,ALACHUA,2006,1.645948e+12,Florida
1,ALACHUA,2007,1.972812e+12,Florida
2,ALACHUA,2008,2.540522e+12,Florida
3,ALACHUA,2009,3.180395e+12,Florida
4,ALACHUA,2010,3.293852e+12,Florida


Renaming the column T_DATE to have it the same everywhere:

In [29]:
df_prescriptions = df_prescriptions.rename({'T_DATE': 'Year', 'BUYER_COUNTY': 'County'}, axis = 1)

Changing the values in County to be capitalized and lowercase to be the same as in the other dataframes:

In [30]:
df_prescriptions['County'] = df_prescriptions['County'].str.lower()
df_prescriptions['County'] = df_prescriptions['County'].str.capitalize()
df_prescriptions.head()

,County,Year,MME,State
0,Alachua,2006,1.645948e+12,Florida
1,Alachua,2007,1.972812e+12,Florida
2,Alachua,2008,2.540522e+12,Florida
3,Alachua,2009,3.180395e+12,Florida
4,Alachua,2010,3.293852e+12,Florida


Checking missing data

In [31]:
df_prescriptions.isna().sum()

County    0
Year      0
MME       0
State     0
dtype: int64

Importing thr second dataset containing the death data

In [32]:
df_deaths = pd.read_csv('https://raw.githubusercontent.com/MIDS-at-Duke/pds2021-opioids-team-three/main/20_intermediate_files/deaths.csv?token=AI3GZD6NJFVWRHJMICW5FSTBRO4Z2')
df_deaths.head()

,County,County Code,Year,Deaths,State
0,Acadia Parish,22001.0,2003.0,11.0,LA
1,Acadia Parish,22001.0,2005.0,23.0,LA
2,Acadia Parish,22001.0,2006.0,19.0,LA
3,Acadia Parish,22001.0,2007.0,19.0,LA
4,Acadia Parish,22001.0,2009.0,11.0,LA


Changing the datatype of Year from float to int to have the same data format for Year in all dataframes

In [33]:
df_deaths["Year"]= df_deaths["Year"].astype('int64')
df_deaths.head()

,County,County Code,Year,Deaths,State
0,Acadia Parish,22001.0,2003,11.0,LA
1,Acadia Parish,22001.0,2005,23.0,LA
2,Acadia Parish,22001.0,2006,19.0,LA
3,Acadia Parish,22001.0,2007,19.0,LA
4,Acadia Parish,22001.0,2009,11.0,LA


Checking missing data

In [34]:
df_deaths.isna().sum()

County         0
County Code    0
Year           0
Deaths         0
State          0
dtype: int64

Importing the third datasset containing census data

In [36]:
df_census = pd.read_csv('https://raw.githubusercontent.com/MIDS-at-Duke/pds2021-opioids-team-three/main/MidSemester/20_intermediates/CountyPopulations.csv?token=AI3GZD36R6FVBM277DEVPB3BRRDSC', index_col= 0)
df_census.head()

,State,County,Year,Population
0,Alabama,Autauga County,2006,49105.0
1,Alabama,Baldwin County,2006,168516.0
2,Alabama,Barbour County,2006,29556.0
3,Alabama,Bibb County,2006,21285.0
4,Alabama,Blount County,2006,55978.0


Checking missing data

In [37]:
df_census.isna().sum()

State         0
County        0
Year          0
Population    6
dtype: int64

### Next step merging
I willbe merging all three datasets into one via a left join on prescription because that is the table where we want all the rows from

---

Because we have multiple dataframes and not only 2 we can't just simply merge. First, we create a list with our three dataframes.

In [46]:
# compile  list of dataframes we want to merge
data_frames = [df_prescriptions, df_deaths, df_census]

Function merging the three dataframes, outer join to not loose any data. Merging on County, Year, State  which can be found in every of the three datasets. 

In [54]:
from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['County', 'Year', 'State'],
                                  how='outer', validate = 'one_to_one'), data_frames)

Looking at the merged data, and the missing data.

In [58]:
df_merged.head()
df_merged.isna().sum()


County             0
Year               0
MME            39345
State              0
County Code    35342
Deaths         35342
Population     11853
dtype: int64

In [49]:
df_merged.describe()

,Year,MME,County Code,Population
count,43267.000000,3.922000e+03,7925.000000,3.141400e+04
mean,2010.306469,2.347498e+12,29500.645552,9.857610e+04
std,3.025097,1.662164e+13,15569.975591,3.169170e+05
min,2003.000000,3.027000e+02,1003.000000,4.000000e+01
25%,2008.000000,7.003393e+09,17019.000000,1.100075e+04
50%,2010.000000,4.510575e+10,32003.000000,2.564650e+04
75%,2013.000000,3.389654e+11,42051.000000,6.643825e+04
max,2015.000000,4.258041e+14,56037.000000,1.017029e+07


Creating a csv file in case we need that

In [52]:
pd.DataFrame.to_csv(df_merged, 'merged_data.csv', sep=',', na_rep='.', index=False)